### Assess model performance post-training

#### Parameters

In [1]:
config = '/home/Thomas/ABC_expeditions_classifier/configs/exp_resnet18.yaml'
split = 'test' # options are "train", "test", and "none"

In [2]:
import torch
import pandas as pd

import yaml

from ct_classifier.train import create_dataloader
from ct_classifier.dataset import CTDataset
from ct_classifier.model import CustomResNet18

cfg = yaml.safe_load(open(config, 'r'))
checkpoint_path="/home/Thomas/ABC_expeditions_classifier/ct_classifier/model_states/200.pt"

# 1. Loading the model

device = cfg['device']

model = CustomResNet18(cfg['num_classes'])

# 2. Grab the latest model (if pre-trained model exists)

state = torch.load(open(checkpoint_path, 'rb'), map_location=device)
model.load_state_dict(state['model'])
model.to(device)
model.eval()

# 3. Prepare the data loader

dl_val = create_dataloader(cfg, split='test')

batch = next(iter(dl_val))



/home/Thomas/miniconda3/envs/ABC_expeditions_classifier/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/Thomas/miniconda3/envs/ABC_expeditions_classifier/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_387311/3704398385.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (

In [17]:
len(dl_val)

110

In [5]:
from torch.nn import Softmax
import numpy as np

prediction_labels = []
prediction_confs = []
true_labels = []
file_list = []

for idx, (data, labels, file_names) in enumerate(dl_val):
  
  data = data.to(device)
  
  # Predict labels on validation data
  
  s = Softmax(dim=1)
  
  with torch.no_grad():
    prediction = model(data)
  
  prediction = s(prediction)
  pred_label = torch.argmax(prediction, dim=1)
  pred_conf = prediction.max(dim = 1).values
  
  prediction_labels.append(pred_label.cpu().numpy())
  prediction_confs.append(pred_conf.cpu().numpy())
  
  # Get ground-truth labels 
  true_labels.append(labels)
  file_list.append(file_names)

prediction_df = {'pred_labels': np.hstack(prediction_labels),
                     "pred_confs" : np.hstack(prediction_confs), 
                     "true_labels": np.hstack(true_labels), 
                     "file_names": np.hstack(file_list)}
    
prediction_df = pd.DataFrame(prediction_df)

In [6]:
len(prediction_df)

13937

In [4]:
import numpy as np

prediction_df.file_names.value_counts()

file_names
62b95746-4eaf-4050-bd83-13f7a7039ee4_frame128_detection0    2
62b95746-4eaf-4050-bd83-13f7a7039ee4_frame16_detection0     2
62b95746-4eaf-4050-bd83-13f7a7039ee4_frame34_detection0     2
62b95746-4eaf-4050-bd83-13f7a7039ee4_frame70_detection0     2
62b95746-4eaf-4050-bd83-13f7a7039ee4_frame98_detection0     2
                                                           ..
5c7e8e8c-36b9-4945-bdb8-9878e10d1f19_frame0_detection0      1
5c7e8e8c-36b9-4945-bdb8-9878e10d1f19_frame22_detection0     1
5c7e8e8c-36b9-4945-bdb8-9878e10d1f19_frame46_detection0     1
5ca31226-a197-4f50-9530-22f536283eb6_frame0_detection0      1
5c663baf-abcd-418b-a371-886436f99032_frame18_detection0     1
Name: count, Length: 13804, dtype: int64

In [20]:
len(prediction_df)

13976

In [7]:
import yaml
from ct_classifier.train import create_dataloader

cfg = yaml.safe_load(open(config, 'r'))

dl_test = create_dataloader(cfg = cfg, split=split)

batch = next(iter(dl_test))



In [8]:
for idx, (data, labels, file_names) in enumerate(dl_test):

    print(len(file_names))

128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128


KeyboardInterrupt: 

In [2]:
from ct_classifier.inference import model_inference

test = model_inference(config=config, checkpoint_path="/home/Thomas/ABC_expeditions_classifier/ct_classifier/model_states/200.pt")

/home/Thomas/miniconda3/envs/ABC_expeditions_classifier/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/Thomas/miniconda3/envs/ABC_expeditions_classifier/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/Thomas/ABC_expeditions_classifier/ct_classifier/inference.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arb

110
110
110
24


ValueError: All arrays must be of the same length